In [ ]:
!pip install folium
!pip install wget

In [ ]:
import folium
import wget
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# Get the count of launches at each site
launch_counts = spacex_df['Launch Site'].value_counts()
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
print(spacex_df.shape)
for index, row in spacex_df.iterrows():
    # Extract coordinates from the current row
    site_coordinate = [row['Lat'], row['Long']]
    # Create a circle at the current site's coordinate with a popup label showing its name
    circle = folium.Circle(site_coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))

    # Create a marker at the current site's coordinate with a icon showing its name
    marker = folium.map.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )

    # Add the circle and marker to the site_map
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

(56, 4)


# Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
#spacex_df.tail(5)
launch_sites_df.tail(5)

,Launch Site,Lat,Long,marker_color
0,CCAFS LC-40,28.562302,-80.577356,red
1,CCAFS SLC-40,28.563197,-80.576820,red
2,KSC LC-39A,28.573255,-80.646895,red
3,VAFB SLC-4E,34.632834,-120.610745,red


Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a MarkerCluster object

In [ ]:
marker_cluster = MarkerCluster()

TODO: Create a new column in launch_sites dataframe called marker_color to store the marker colors based on the class value

In [ ]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Appliquer la fonction assign_marker_color à chaque valeur dans la colonne 'class'
# et créer une nouvelle colonne 'marker_color' avec les couleurs retournées
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
print(spacex_df.shape)
spacex_df.tail(5)


(56, 5)


,Launch Site,Lat,Long,class,marker_color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


In [ ]:
launch_sites_df.tail(5)

,Launch Site,Lat,Long,marker_color
0,CCAFS LC-40,28.562302,-80.577356,red
1,CCAFS SLC-40,28.563197,-80.576820,red
2,KSC LC-39A,28.573255,-80.646895,red
3,VAFB SLC-4E,34.632834,-120.610745,red


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
print(spacex_df.shape)
for index, record in launch_sites_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=record['Launch Site']
    )
    marker_cluster.add_child(marker)

site_map


(56, 5)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
# find coordinate of the closet coastline
coastline_lat= 28.56257
coastline_lon= -80.56786
launch_site_lat=28.562302
launch_site_lon=	-80.577356
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

print(distance_coastline)

0.9281695198317069


In [ ]:
coordinate = [coastline_lat, coastline_lon]
launch_site_coo = [launch_site_lat, launch_site_lon]
coordinates=[coordinate,launch_site_coo]
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )

In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [ ]:
# A railway map [28.5731,-80.58523] train
railway_lat= 28.5731
railway_lon= -80.58523
launch_site_lat=28.562302
launch_site_lon=	-80.577356
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

print(distance_railway)
coordinate = [railway_lat, railway_lon]
launch_site_coo = [launch_site_lat, launch_site_lon]
coordinates=[coordinate,launch_site_coo]
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
       )
   )

1.4262554847520785


In [ ]:
coordinate = [railway_lat, railway_lon]
launch_site_coo = [launch_site_lat, launch_site_lon]
coordinates=[coordinate,launch_site_coo]
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
       )
   )

In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [ ]:
# A [28.57011,-80.5722] highway
highway_lat= 28.57011
highway_lon= -80.5722
launch_site_lat=28.562302
launch_site_lon=	-80.577356
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

print(distance_highway)
coordinate = [highway_lat, highway_lon]
launch_site_coo = [launch_site_lat, launch_site_lon]
coordinates=[coordinate,launch_site_coo]
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
       )
   )

coordinate = [highway_lat, highway_lon]
launch_site_coo = [launch_site_lat, launch_site_lon]
coordinates=[coordinate,launch_site_coo]
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
       )
   )

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

1.0039728438093702


In [ ]:
# A [28,40151,-80,60466] city
city_lat= 28.40151
city_lon= -80.60466
launch_site_lat=28.562302
launch_site_lon=	-80.577356
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

print(distance_city)
coordinate = [city_lat, city_lon]
launch_site_coo = [launch_site_lat, launch_site_lon]
coordinates=[coordinate,launch_site_coo]
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
       )
   )

coordinate = [city_lat, city_lon]
launch_site_coo = [launch_site_lat, launch_site_lon]
coordinates=[coordinate,launch_site_coo]
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
       )
   )

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

18.082986263407903


In [ ]:
railway_lat= 28.5731
railway_lon= -80.58523

highway_lat= 28.57011
highway_lon= -80.5722

city_lat= 28.40151
city_lon= -80.60466
# Create a marker at the specified coordinates
marker = folium.Marker(
    location=[railway_lat, railway_lon],
    popup="railway",
    icon=folium.Icon(color="blue", icon="info-sign"),
)

# Add the marker to the site_map
site_map.add_child(marker)

marker = folium.Marker(
    location=[highway_lat, highway_lon],
    popup="highway",
    icon=folium.Icon(color="blue", icon="info-sign"),
)

# Add the marker to the site_map
site_map.add_child(marker)

marker = folium.Marker(
    location=[city_lat, city_lon],
    popup="city",
    icon=folium.Icon(color="blue", icon="info-sign"),
)

# Add the marker to the site_map
site_map.add_child(marker)

# Display the map
site_map

